In [1]:
import keras
from keras.layers import GlobalAveragePooling2D
from keras import layers, Input, optimizers, models
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
img_size = (224, 224)
BATCH_SIZE = 1
NUM_EPOCHS = 100
DATAPATCH = '../2019_endoscopy_image/'
classes = ['normal', 'abnormal']

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(preprocess_input,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,                     
         fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(
    directory=DATAPATCH+'train', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
test_datagen = ImageDataGenerator(preprocess_input)
valid_batches = test_datagen.flow_from_directory(
    directory=DATAPATCH+'test', target_size=img_size, batch_size=BATCH_SIZE, class_mode='binary', classes=classes)
input_tensor = Input(shape=(124, 124, 3), dtype='float32', name='input')

W0905 19:30:16.130804 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Found 2561 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [3]:
pre_VGG = VGG16(weights='imagenet', include_top=False,
                input_shape=(224, 224, 3))
pre_VGG.summary()

W0905 19:30:16.167435 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 19:30:16.170855 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 19:30:16.198806 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0905 19:30:16.434956 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0905 1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
# add a global spatial average pooling layer
x = pre_VGG.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = layers.Dense(2048, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model1 = Model(inputs=pre_VGG.input, outputs=predictions)
model1.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9),
                 loss='binary_crossentropy', metrics=['accuracy'])
allModel.summary()

W0905 19:30:17.316863 140264612206400 deprecation_wrapper.py:119] From /home/leeon/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0905 19:30:17.335225 140264612206400 deprecation.py:323] From /home/leeon/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 27,692,098
Trainable params: 12,977,410
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# Specify the Log Directory
import datetime as dt

# seriously? STRingFormatTIME? You gottabe kidding me
logdir = "logs/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [5]:
history = model1.fit_generator(train_data, epochs=NUM_EPOCHS,
                                 validation_steps=test_data.samples,
                                 steps_per_epoch=train_data.samples,
                                 validation_data=test_data,
                                 callbacks=[tensorboard_callback]).history

Epoch 1/100


ValueError: Error when checking target: expected dense_3 to have shape (2,) but got array with shape (1,)

In [ ]:
loss = history['loss']
val_loss = history['val_loss']
acc = history['acc']
val_acc = history['val_acc']
epochs = range(1, len(loss)+1)
epochs2 = range(1,len(val_loss)+1)
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss/Acc')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='train loss')
plt.plot(epochs2, val_loss, 'c', label='Val loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(epochs, acc, 'g', label='train acc')
plt.plot(epochs2, val_acc, 'b', label='Val acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
test_batches = train_datagen.flow_from_directory(DATAPATCH + 'test',
                                                  target_size=img_size,
                                                  class_mode='binary',
                                                  batch_size=BATCH_SIZE,
                                                 classes= classes ,
                                                 shuffle=False)
result = model1.evaluate_generator(generator=test_batches,steps = test_batches.samples)
print(result)